# BFGS

## Description
The Broyden-Fletcher-Goldfarb-Shanno aka BFGS method is a second-order iterative optimization method.

### Quasi-Newton Methods
The BFGS method is referred to as Quasi-Newton in reference to the fact that unlike Newton's method which uses an explicit Hessian matrix, these methods approximate the Hessian. 

$$
x_{k+1} = x_k - \alpha_k B_k^{-1} \nabla f(x_k)
$$

where
* $B_k$ is approxmation to Hessian
* $\alpha_k$ is obtained from line search

Secant updating methods have superlinear convergence ($1 < r < 2$).
* Slower to converge than Newton's method, but cost-per-iteration is less.

### BFGS Algorithm
1. Start with some initial guess $x_0$ and approximate Hessian $B_0 = I$.
2. Solve $B_k s_k = -\nabla f(x_k)$ for $s_k$ or use a line search (described below) to find $s_k$.
3. Compute $x_{k+1} = x_k + s_k$.
4. Compute the difference in gradients $y_k = \nabla f(x_{k+1}) - \nabla f(x_k)$.
5. Update approximate Hessian.
$$
B_{k+1} = B_k + \frac{y_k y_k^T}{y_k^T s_k} - \frac{B_k s_k s_k^T B_k}{s_k^T B_k s_k}
$$
6. Repeat from step 2 until some stopping criteria is reached.

### References
> Michael T. Heath. 2018. Scientific Computing: An Introductory Survey, Revised Second Edition. SIAM-Society for Industrial and Applied Mathematics, Philadelphia, PA, USA.

In [1]:
# The numpy interface of autograd wraps all numpy ops with autodiff.
import autograd.numpy as np

from autograd import grad

## Line Search

A line search is used to find the distance along the descent direction of the next step of the optimization.  The scalar multiple $\alpha$ along the descent direction $d$ is found by minimizing the function below.

$$
\underset{\alpha}{\text{minimize}} f(x_k + \alpha d)
$$
where
* $f(...)$ is the function to minimize
* $x_k$ is the current solution
* $\alpha$ is a scalar 
* $d$ is a vector that describes the descent direction of the function

For first-order optimization problems the descent direction is given by the negative gradient $-\nabla f(x_k)$.

For second-order optimization problems the descent direction is given by by the product of the negative gradient and Hessian $-\nabla f(x_k) H_k$.

In [2]:
def bracket_minimum(fx, x, s, k):
    """
    bracket_minimum returns interval [a,b] that brackets mininum of fx
    
    Parameters
    ----------
    fx : function
        function
    x : numpy.ndarray
        starting position around which bracket is found
    s : numpy.ndarray
        initial step size separating [a,b]
    k : float
        scaling factor applied to step size at each iteration

    Returns
    -------
    numpy.ndarray
        lower bound of bracket interval
    numpy.ndarray
        upper bound of bracket interval
    """
    a, fxa = x, fx(x)
    b, fxb = x + s, fx(x + s)
    if fxb > fxa:  # Invariant: a < b.
        a, b = b, a
        fxa, fxb = fxb, fxa
        s = -s
    while True:
        c, fxc = b + s, fx(b + s)
        if fxc > fxb:
            break
        a, fxa, b, fxb = b, fxb, c, fxc
        s = s * k
    if a < c:
        return a, c
    return c, a


def goldensection(fx, a, b, tol):
    """
    goldensection returns the minimum of fx over some interval [a,b]

    Parameters
    ----------
    fx : function
        function
    a : numpy.ndarray
        lower bound of interval that brackets minimum of fx
    b : numpy.ndarray
        upper bound of interval that brackets minimum of fx
    tol : float
        convergence threshold

    Returns
    -------
    numpy.ndarray
        point along interval [a,b] where fx is minimum
    """
    tau = (np.sqrt(5) - 1.) / 2.  # Golden ratio - 1.
    x1, x2 = a + (1. - tau) * (b - a), a + tau * (b - a)
    fx1, fx2 = fx(x1), fx(x2)
    
    while (b - a) > tol:
        if fx1 < fx2:
            b = x2
            # Treat x1 as the new x2.
            x2, fx2 = x1, fx1
            # Compute new x1.
            x1 = a + (1. - tau) * (b - a)
            fx1 = fx(x1)
        else:
            a = x1
            # Treat x2 as the new x1.
            x1, fx1 = x2, fx2
            # Compute new x2.
            x2 = a + tau * (b - a)
            fx2 = fx(x2)
    
    return x1


def line_search(fx, d, xk, tol):
    """
    line_search returns the offset from xk where fx is minimum

    Parameters
    ----------
    fx : function
        function
    d : numpy.ndarray
        descent direction of fx, typically negative gradient at xk
    xk : numpy.ndarray
        starting position of search
    tol : float
        convergence threshold

    Returns
    -------
    alpha : float
        scalar multiple along descent direction where fx is minimum
    numpy.ndarray
        position xk where fx is minimum
    """

    # Objective function to minimize.
    fobj = lambda alpha: fx(xk + alpha * d)
    alpha0 = 1e-6

    # Find interval [a,b] closest to alpha0 that brackets the minimum.
    a, b = bracket_minimum(fobj, alpha0, s=1e-2, k=2.)

    # Find minimum within the bracket [a,b].
    alphak = goldensection(fobj, a, b, tol)

    # Position where fx is minimum.
    xkmin  = xk + alphak * d
    
    return alphak, xkmin

## BFGS Method

In [3]:
def bfgs(fx, gradfx, x0, tol, maxiter):
    """
    bfgs returns the point xk where fx is minimum

    Parameters
    ----------
    fx : function
        function to minimize
    gradfx : function
        gradient of function to minimize
    x0 : numpy.ndarray
        initial guess for xk
    tol : float
        convergence threshold
    maxiter : int
        maximum number of iterations

    Returns
    -------
    numpy.ndarray
        vector xk where fx is minimum
    numpy.ndarray
        position and value history
        [[x0, fx(x0), gradfx(x0)],
         [x1, fx(x1), gradfx(x1)],...]
    """

    xk, gradfxk, Bk = x0, gradfx(x0), np.eye(x0.size)

    # Save current and minimum position and value to history.
    steps = np.zeros((maxiter, (x0.size*2)+1))
    steps[0,:] = np.hstack((x0, fx(x0), gradfxk))

    # Repeat up to maximum number of iterations.
    for k in range(1,maxiter):

        # Stop iteration when gradient is near zero.
        if np.linalg.norm(gradfxk) < tol:
            steps = steps[:-(maxiter-k),:]
            break

        # Solve Bk*sk = -grad(xk) for sk.
        try:
            sk = np.linalg.solve(Bk, -1. * gradfxk)
        except np.linalg.LinAlgError:
            # Perform line search to find alphak.
            normd = -np.dot(Bk, gradfxk)
            normd = normd / np.linalg.norm(normd)
            alphak, xkp1 = line_search(fx, normd, xk, tol)
            sk = xkp1 - xk

        # Update xk and evaluate gradient at new value of xk.
        xk = xk + sk
        gradfxk1 = gradfx(xk)

        # Compute difference in gradients.
        yk = gradfxk1 - gradfxk

        # Update approximate Hessian.
        term1 = np.outer(yk, yk.T) / np.dot(yk.T, sk)
        term2a = np.dot(np.dot(Bk, np.outer(sk, sk.T)), Bk)
        term2b = np.dot(np.dot(sk.T, Bk), sk)
        Bk = Bk + term1 - (term2a / term2b)

        # Update the gradient at xk.
        gradfxk = gradfxk1

        # Save iteration history.
        steps[k,:] = np.hstack((xk, fx(xk), gradfxk))

    return xk, steps

## Test Function: Rosenbrock Function

In [4]:
def rosenbrock(x):
    """
    rosenbrock evaluates Rosenbrock function at vector x

    Parameters
    ----------
    x : array
        x is a D-dimensional vector, [x1, x2, ..., xD]

    Returns
    -------
    float
        scalar result
    """
    D = len(x)
    i, iplus1 = np.arange(0,D-1), np.arange(1,D)
    return np.sum(100*(x[iplus1] - x[i]**2)**2 + (1-x[i])**2)

## Solution to Rosenbrock Function

In [5]:
fx, gradfx = rosenbrock, grad(rosenbrock)
x0, tol, maxiter = np.array([-1.,-1.]), 1e-2, 1000
xk, steps = bfgs(fx, gradfx, x0, tol, maxiter)

print("x0               :", x0)
print("rosenbrock f(w0) :", rosenbrock(x0))
print("----------------------------------")
print("xk               :", xk)
print("rosenbrock f(xk) :", rosenbrock(xk))
print("nsteps           :", len(steps))
print("norm(gradfx)     :", np.linalg.norm(steps[-1,3:]))

x0               : [-1. -1.]
rosenbrock f(w0) : 404.0
----------------------------------
xk               : [0.9999072  0.99982128]
rosenbrock f(xk) : 1.3326100865859894e-08
nsteps           : 121
norm(gradfx)     : 0.0032376948029769173


## Test Function: Goldstein-Price Function

In [6]:
def goldstein_price(x):
    """
    goldstein_price evaluates Goldstein-Price function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    a = (x[0] + x[1] + 1)**2
    b = 19 - 14*x[0] + 3*x[0]**2 - 14*x[1] + 6*x[0]*x[1] + 3*x[1]**2
    c = (2*x[0] - 3*x[1])**2
    d = 18 - 32*x[0] + 12*x[0]**2 + 48*x[1] - 36*x[0]*x[1] + 27*x[1]**2
    return (1. + a*b) * (30. + c*d)

## Solution to Goldstein-Price Function

In [7]:
fx, gradfx = goldstein_price, grad(goldstein_price)
x0, tol, maxiter = np.array([-1.0,-1.5]), 1e-2, 1000
xk, steps = bfgs(fx, gradfx, x0, tol, maxiter)

print("x0                    :", x0)
print("goldstein_price f(w0) :", goldstein_price(x0))
print("----------------------------------")
print("xk                    :", xk)
print("goldstein_price f(xk) :", goldstein_price(xk))
print("nsteps                :", len(steps))
print("norm(gradfx)          :", np.linalg.norm(steps[-1,3:]))

x0                    : [-1.  -1.5]
goldstein_price f(w0) : 1595.41015625
----------------------------------
xk                    : [ 1.40617001e-05 -9.99995623e-01]
goldstein_price f(xk) : 3.000000044809572
nsteps                : 95
norm(gradfx)          : 0.006186613015289871
